In [3]:
from pyomo.environ import (
    SolverFactory,
    AbstractModel,
    Param,
    Set,
    Var,
    Objective,
    Constraint,
)
from pyomo.core import NonNegativeIntegers, PositiveIntegers, PositiveReals, Any, NonNegativeReals, Binary  # type: ignore
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Tuple

Point = Tuple[float, float]
Points = list[Point]

solver = SolverFactory("glpk")


## Storage Problem/System Summary

$$
\begin{aligned}
  &\text{min} \sum_{t=0}^T w(t) \\
  &\text{s.t.} \\
  &t = \{1, 2, ..., T\} \text{ as time points} \\
  &\text{$s(t)$ as reservoir storage, $a(t)$ as surface area, $r(t)$ as rainfall, } \\
  &\text{$e(t)$ as evaporation, $q(t)$ as pumping amount, $w(t)$ as overflow at or over time $t$} \\
  &s(t + 1) = s(t) + r(t) - e(t)- q(t) - w(t), \forall t \\
  &\text{$s(t) \leq \bar S$ for maximum capacity, $a(t) \leq \bar A$ as maximum surface area, and $q(t) \leq \bar Q$ as maximum pumping rate} \\
  &r(t) = A_c p(t) \text{ where $A_c$ is a constant catchment area and $p(t)$ is precipitation} \\
  &e(t) = a(t) e_0(t) \text{ where $e_0(t)$ is the evaporation coefficient at time $t$} \\
  &s(t) = c_1 a(t) ^ {c_2} \text{(power law relationship with constants)}
\end{aligned}
$$


In [27]:
model = AbstractModel()

model.T = Set(domain=NonNegativeIntegers)  # Time

# Constant Input Parameters
c1 = Param(domain=PositiveReals)  # Scalar constant c1
c2 = Param(domain=PositiveReals)  # Scalar constant c2
A_c = Param(domain=PositiveReals)  # Catchment area in square meters
S_max = Param(domain=PositiveReals)  # Maximum storage capacity
A_max = Param(domain=PositiveReals)  # Maximum free surface area
Q_max = Param(domain=NonNegativeReals)  # Upper limit for pumping rate

# Time-varying Input Parameters
model.P = Param(model.T, domain=NonNegativeReals)  # Precipitation
model.E_coeff = Param(model.T, domain=NonNegativeReals)  # Evaporation coefficients

# Calculated Parameters
model.R = Param(  # Rainfall
    model.T,
    default=lambda model, t: model.A_c * model.P[t],
    within=NonNegativeReals,
)

# Optimizeable Variables
model.S = Var(model.T, domain=NonNegativeReals)  # Storage
model.E = Var(model.T, domain=NonNegativeReals)  # Evaporation
model.A = Var(model.T, domain=NonNegativeReals)  # Surface area
model.Q = Var(model.T, domain=NonNegativeReals)  # Pumping
model.W = Var(model.T, domain=NonNegativeReals)  # Overflow

# Objective
model.min_overflow = Objective(expr=lambda model: sum(model.W[t] for t in model.T))

# Constraints
model.upper_S = Constraint(model.T, rule=lambda model, t: model.S[t] <= model.S_max)

model.upper_A = Constraint(model.T, rule=lambda model, t: model.A[t] <= model.A_max)

model.upper_Q = Constraint(model.T, rule=lambda model, t: model.Q[t] <= model.Q_max)

model.evaporation = Constraint(
    model.T,
    rule=lambda model, t: model.E[t] == model.E_coeff[t] ** model.A[t],
)

model.storage = Constraint(
    model.T, rule=lambda model, t: model.S[t] == model.c1 * model.A[t] + model.c2
)

model.dynamics = Constraint(
    model.T,
    rule=lambda model, t: model.S[t + 1]
    == model.S[t] + model.R[t] - model.E[t] - model.Q[t] - model.W[t],
)

In [28]:
len_T = 12

data = {
    None: {
        "T": {None: list(range(len_T))},
        "c1": {None: 0.5},
        "c2": {None: 0.5},
        "A_c": {None: 100},
        "S_max": {None: 100},
        "A_max": {None: 100},
        "Q_max": {None: 100},
        "P": {i: p for i, p in enumerate([0.5] * len_T)},
        "E_coeff": {i: e_coeff for i, e_coeff in enumerate([0.7] * len_T)},
    }
}

instance = model.create_instance(data)
# results = solver.solve(instance)
# instance.pprint()

ERROR: Rule failed when generating expression for Constraint upper_S with
    index 0: AttributeError: 'AbstractModel' object has no attribute 'S_max'
ERROR: Constructing component 'upper_S' from data=None failed:
        AttributeError: 'AbstractModel' object has no attribute 'S_max'


AttributeError: 'AbstractModel' object has no attribute 'S_max'